In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm

from dyn_fed.data.mnist import MNist
import dyn_fed as df

%load_ext autoreload
%autoreload
%config Completer.use_jedi=False

In [2]:
tf.executing_eagerly()

True

In [3]:
X_train, y_train, X_test, y_test = df.data.mnist.load_data()

In [4]:
X_train.shape

(60000, 28, 28)

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

Non customizable training loop way

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    # tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation="sigmoid")
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [24]:
%%time
model.fit(train_dataset, epochs=10)

Train for 469 steps
Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 2.1880 - sparse_categorical_accuracy: 0.4191A: 0s - loss: 2.2406 - sparse_categorical_accuracy: 0
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 2.0214 - sparse_categorical_accuracy: 0.7168
Epoch 3/10
469/469 [==============================] - 1s 1ms/step - loss: 1.9301 - sparse_categorical_accuracy: 0.7784
Epoch 4/10
469/469 [==============================] - 1s 1ms/step - loss: 1.8754 - sparse_categorical_accuracy: 0.8005
Epoch 5/10
469/469 [==============================] - 1s 1ms/step - loss: 1.8390 - sparse_categorical_accuracy: 0.8113
Epoch 6/10
469/469 [==============================] - 1s 1ms/step - loss: 1.8130 - sparse_categorical_accuracy: 0.8178
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 1.7932 - sparse_categorical_accuracy: 0.8226
Epoch 8/10
469/469 [==============================] - 1s 1ms/step - loss: 1.7776 - sparse_cate

In [25]:
model.evaluate(test_dataset)

79/79 [==============================] - 0s 1ms/step - loss: 1.7411 - sparse_categorical_accuracy: 0.8446


[1.7410907488835008, 0.8446]

Customizable training loop

In [26]:
# Define logistic regression model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    # tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation="sigmoid")
])

# Define optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# Define loss function
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [27]:
@tf.function
def train_loop(x, y):

    # Calculate gradients
    with tf.GradientTape() as t:
        # training=training is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(x, training=True)
        loss = loss_func(y, predictions)

    grads = t.gradient(loss, model.trainable_variables)

    # Optimize the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Track progress
    epoch_loss_avg(loss)

    # Compare predicted label to actual
    epoch_accuracy.update_state(y, predictions)
    
#     return loss, predictions

In [28]:
%%time
train_loss_results = []
train_accuracy_results = []
epochs = 50
n_batches = len(list(train_dataset))

for epoch in tqdm(np.arange(epochs)):
    
    for x, y in tqdm(train_dataset, total=n_batches):
        train_loop(x, y)

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    print(
        "Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(
            epoch,
            epoch_loss_avg.result(),
            epoch_accuracy.result()
        )
    )
    
    # Clear the current state of the metrics
    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()
    # valid_loss.reset_states(), valid_acc.reset_states()
    
    

Epoch 000: Loss: 2.170, Accuracy: 46.763%


Epoch 001: Loss: 2.010, Accuracy: 73.305%


Epoch 002: Loss: 1.924, Accuracy: 78.222%


Epoch 003: Loss: 1.872, Accuracy: 80.215%


Epoch 004: Loss: 1.837, Accuracy: 81.267%


...

Epoch 045: Loss: 1.650, Accuracy: 86.950%


Epoch 046: Loss: 1.649, Accuracy: 86.973%


Epoch 047: Loss: 1.648, Accuracy: 87.012%


Epoch 048: Loss: 1.647, Accuracy: 87.047%


Epoch 049: Loss: 1.646, Accuracy: 87.078%

CPU times: user 35.8 s, sys: 6.43 s, total: 42.2 s
Wall time: 30.3 s


## Other

In [265]:
import functools

In [269]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                print("Something")
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [270]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

In [271]:
greet("Sashlin")

Something
Hello Sashlin
Something
Hello Sashlin
Something
Hello Sashlin
Something
Hello Sashlin


In [329]:
class config(dict):
    MARKER = object()

    def __init__(self, value=None):
        if value is None:
            pass
        elif isinstance(value, dict):
            for key in value:
                self.__setitem__(key, value[key])
        else:
            raise TypeError('expected dict')

    def __setitem__(self, key, value):
        if isinstance(value, dict) and not isinstance(value, config):
            value = config(value)
        super(config, self).__setitem__(key, value)

    def __getitem__(self, key):
        found = self.get(key, config.MARKER)
        if found is config.MARKER:
            found = config()
            super(config, self).__setitem__(key, found)
        return found

    __setattr__, __getattr__ = __setitem__, __getitem__